In [3]:
#TODO visualize the train valid test split in adjacency matrix 

import os
import sys
sys.path.insert(0, '/hkfs/work/workspace/scratch/cc7738-benchmark_tag/TAPE_chen/core')

from data_utils.load_data_nc import load_graph_cora, load_graph_pubmed, load_tag_arxiv23, load_graph_ogbn_arxiv
from torch_geometric import seed_everything
from graphgps.encoder.seal import get_pos_neg_edges, extract_enclosing_subgraphs, k_hop_subgraph, construct_pyg_graph, do_edge_split
import copy

/home/hk-project-test-p0021478/cc7738/anaconda3/envs/EAsF/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
name = 'cora'
val_pct = 0.15
test_pct = 0.05

In [5]:
seed_everything(0)
if name == 'pubmed':
    data = load_graph_pubmed(False)
elif name == 'cora':
    data, _ = load_graph_cora(False)
elif name == 'arxiv_2023':
    data, _ = load_tag_arxiv23()
elif name == 'ogbn-arxiv':
    data = load_graph_ogbn_arxiv(False)
splits = do_edge_split(copy.deepcopy(data), val_pct,  test_pct)

Processing...
Done!
/home/hk-project-test-p0021478/cc7738/anaconda3/envs/EAsF/lib/python3.8/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [ ]:
# visualize it using adajcency plot visualization/..adj

In [ ]:
# visualize them using CN distribution using visualization/..functions

In [ ]:
#  these funcs are not from pytorch.geometric, visualize and calculate some graph statistic such as degree distribution
# avg, avg^2
extract_enclosing_subgraphs
k_hop_subgraph